In [48]:
#import libraries
from pymongo import MongoClient
from pprint import pprint
from bson.json_util import dumps
import redis
import json
import time
import pandas as pd
import numpy as np
import csv, redis, json
import sys

In [ ]:
def prepareModel(columns,rowdata):
    model ={}
    i = 0
    while i < len(rowdata):
        model[columns[i]] = rowdata[i]
        i += 1     
    return model

In [ ]:
# read dataset
def read_csv_data(path):
    columns = []
    dataModel = []
    with open(path, encoding="utf8") as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            if line_count == 0:
                print(row)
                #get colums
                for i in range(len(row)):
                    columns.append(row[i])
                line_count += 1
            else:
                dataModel.append(prepareModel(columns,row))
                line_count += 1
    return dataModel
    

In [ ]:
def store_data_to_redis(conn, data):
    for i in data:
        conn.setnx(i[0], i[1])
    return data

In [ ]:
data = read_csv_data("../data/trump.csv")
data
#create Redis connection
#conn = redis.Redis(host='mycache.xpz4sa.ng.0001.use1.cache.amazonaws.com',port=6379)
#print("Redis connected....")
#print (json.dumps(store_data(conn, data)))
